In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import urllib.parse

# ---------------- SQL Server Connection ----------------
SERVER = 'localhost'
DATABASE = 'PakWheelsDB'
DRIVER = '{ODBC Driver 18 for SQL Server}'

params = urllib.parse.quote_plus(
    f"DRIVER={DRIVER};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"Trusted_Connection=yes;"
    f"TrustServerCertificate=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [2]:
# ---------------- Create Tables in SQL ----------------
with engine.begin() as conn:
    conn.execute(text("""
        IF OBJECT_ID('Analysis') IS NOT NULL DROP TABLE Analysis;
        CREATE TABLE Analysis (
            Id INT IDENTITY(1,1) PRIMARY KEY,
            MetricName NVARCHAR(200),
            MetricValue NVARCHAR(200),
            Description NVARCHAR(500),
            InsertedAt DATETIME2 DEFAULT GETDATE()
        );
    """))

    conn.execute(text("""
        IF OBJECT_ID('DataFrameAnalysis') IS NOT NULL DROP TABLE DataFrameAnalysis;
        CREATE TABLE DataFrameAnalysis (
            Id INT IDENTITY(1,1) PRIMARY KEY,
            ColumnName NVARCHAR(200),
            StatType NVARCHAR(100),
            StatValue NVARCHAR(200),
            InsertedAt DATETIME2 DEFAULT GETDATE()
        );
    """))

print("Tables created successfully!")


Tables created successfully!


In [3]:
df_sql = pd.read_sql("SELECT * FROM Cars", engine)


In [4]:
mean_price = round(df_sql['PricePKR'].mean()/100000, 2)
corr_year = round(df_sql['Year'].corr(df_sql['PricePKR']), 3)
corr_mileage = round(df_sql['Mileage'].corr(df_sql['PricePKR']), 3)

analysis_df = pd.DataFrame([
    {"MetricName": "Mean Price (Lacs)", "MetricValue": mean_price, "Description": "Average vehicle price"},
    {"MetricName": "Year-Price Correlation", "MetricValue": corr_year, "Description": "Correlation between model year and price"},
    {"MetricName": "Mileage-Price Correlation", "MetricValue": corr_mileage, "Description": "Correlation between mileage and price"},
])

analysis_df.to_sql("Analysis", engine, if_exists="append", index=False)
print("Inserted analysis results successfully!")


Inserted analysis results successfully!


In [5]:
stats_list = []

for col in ["Year", "Mileage", "PricePKR"]:
    stats_list.append({"ColumnName": col, "StatType": "mean", "StatValue": df_sql[col].mean()})
    stats_list.append({"ColumnName": col, "StatType": "min", "StatValue": df_sql[col].min()})
    stats_list.append({"ColumnName": col, "StatType": "max", "StatValue": df_sql[col].max()})

df_stats = pd.DataFrame(stats_list)

df_stats.to_sql("DataFrameAnalysis", engine, if_exists="append", index=False)
print("Inserted DataFrame statistics successfully!")


Inserted DataFrame statistics successfully!


In [6]:
print("Analysis Table:")
print(pd.read_sql("SELECT TOP 5 * FROM Analysis ORDER BY InsertedAt DESC", engine))

print("\nDataFrameAnalysis Table:")
print(pd.read_sql("SELECT TOP 5 * FROM DataFrameAnalysis ORDER BY InsertedAt DESC", engine))


Analysis Table:
   Id                 MetricName MetricValue  \
0   3  Mileage-Price Correlation       -0.43   
1   2     Year-Price Correlation       0.529   
2   1          Mean Price (Lacs)       38.37   

                                Description              InsertedAt  
0     Correlation between mileage and price 2025-11-16 23:03:53.160  
1  Correlation between model year and price 2025-11-16 23:03:53.160  
2                     Average vehicle price 2025-11-16 23:03:53.160  

DataFrameAnalysis Table:
   Id ColumnName StatType StatValue                 InsertedAt
0   6    Mileage      max    295866 2025-11-16 23:04:02.303333
1   5    Mileage      min     41000 2025-11-16 23:04:02.303333
2   4    Mileage     mean    107595 2025-11-16 23:04:02.303333
3   3       Year      max      2022 2025-11-16 23:04:02.303333
4   2       Year      min      1992 2025-11-16 23:04:02.303333
